# Iterated local search (ILS)

In this notebook is the clean implementation of the ILS algorithm with an example of use, it is important to mention that instances must be in the same directory of the notebook.

In [1]:
import numpy as np
import copy
import random
import pandas as pd
import math
import time
import coord20_5_1 #import required instances...
import coord20_5_1b
import coord20_5_2
import coord20_5_2b

In [2]:
#Example of use...
Data=[coord20_5_1, coord20_5_1b, coord20_5_2, coord20_5_2b]

#Note... this data was used for the conducted DoE
#Data DoE
#Data=[coord20_5_1, coord50_5_2, coord100_5_1, coord200_10_3, coordGaspelle4, coordChrist50]

In [3]:
#================================================Functions required=================================================
#Search for information that is required in the ranking function...
def buscar_info(Ob,Db,Sb):
    valor=Ob[Db][Sb]
    return valor

#Ranking of the attractiveness of potential depots
def ranking(Ob,Sb):
    l=[]
    for i in D:
        l.append([buscar_info(O,i,Sb),i])
    l.sort(key=lambda x: x[0])
    i=0
    while i<len(l):
        l[i][0]=i+1
        i+=1
    l.sort(key=lambda x: x[1])
    l2=[]
    for i in l:
        l2.append(i[0])
    return l2

#Intitial open depots
def AbrirDepotsSemilla(mdepots, frec, DepotsAbiertos):
    while np.sum(DepotsAbiertos)<mdepots:
        dado=np.random.rand()
        i=1
        while dado>frec[i]:
            i+=1
        i-=1
        if DepotsAbiertos[i]==1:
            AbrirDepotsSemilla(mdepots, frec, DepotsAbiertos)
        else:
            DepotsAbiertos[i]=1
    return DepotsAbiertos

#Delete repeated data
def EliminarRepetidos(lista):
    data=[]
    for item in lista:
        if item not in data:
            data.append(item)
    return data
#================================================CODE====================================================

#ranking of depots
def depots(O,C,D,hc,d,K,Q,S):
    #================================================REQUIRED LISTS=========================================
    lista_ranking=[]
    lista_puntaje=[]
    depots_abiertos=[]
    for i in D:
        lista_puntaje.append(0)
        depots_abiertos.append(0)

    depots_abiertos2=depots_abiertos.copy()
    depots_abiertos3=depots_abiertos.copy()
    depotsSemilla1=depots_abiertos.copy()
    depotsSemilla2=depots_abiertos.copy()
    depotsSemilla3=depots_abiertos.copy()
    for i in S:
        lista_ranking=ranking(O,i)
        for h in D:
            lista_puntaje[h]+=lista_ranking[h]

    #----------------------------------------HEURISTIC1--------------------------------------------
    #score for opening cost
    lista_puntaje2=lista_puntaje.copy()
    i=0
    while i<len(D):
        lista_puntaje[i]=lista_puntaje[i]*hc[i]
        i+=1

    promedio=np.mean(lista_puntaje)
    desv=np.std(lista_puntaje)
    criterio=promedio-desv/4*3

    #opening criterion
    for i in D:
        if lista_puntaje[i]<criterio:
            depots_abiertos[i]=1

    #critical case without opening
    if depots_abiertos.count(0)==len(D):
        depots_abiertos[lista_puntaje.index(min(lista_puntaje))]=1

    #------------------------------------HEURISTIC2-------------------------------------------------
    #score plus opening cost
    i=0
    while i<len(D):
        lista_puntaje2[i]=lista_puntaje2[i]*np.mean(O)+hc[i]
        i+=1

    promedio2=np.mean(lista_puntaje2)
    desv2=np.std(lista_puntaje2)
    criterio2=promedio2-desv2*np.random.rand()#/4*3

    #opening criterion
    for i in D:
        if lista_puntaje2[i]<criterio2:
            depots_abiertos2[i]=1

    #-----------------------------------HEURISTIC3-----------------------------------------------------
    lista_puntaje3=[]
    i=0

    while i < len(O):
        lista_puntaje3.append(np.mean(O[i])*len(S)/K+hc[i])
        i+=1

    promedio3=np.mean(lista_puntaje3)
    desv3=np.std(lista_puntaje3)
    criterio3=promedio3-desv3/4*3

    for i in D:
        if lista_puntaje3[i]<criterio3:
            depots_abiertos3[i]=1


    #----------------------------------CREATION OF SEEDS FOR ROUTES---------------------------------

    DepotsPorAbrir1=np.sum(depots_abiertos)
    DepotsPorAbrir2=np.sum(depots_abiertos2)
    DepotsPorAbrir3=np.sum(depots_abiertos3)

    lista_puntaje/=np.sum(lista_puntaje)
    lista_puntaje2/=np.sum(lista_puntaje2)
    lista_puntaje3/=np.sum(lista_puntaje3)

    lista_puntaje-=1
    lista_puntaje2-=1
    lista_puntaje3-=1

    lista_puntaje*=-1
    lista_puntaje2*=-1
    lista_puntaje3*=-1

    lista_puntaje/=np.sum(lista_puntaje)
    lista_puntaje2/=np.sum(lista_puntaje2)
    lista_puntaje3/=np.sum(lista_puntaje3)

    frecAcum1=[0]
    frecAcum2=[0]
    frecAcum3=[0]

    i=0
    while i<len(lista_puntaje):
        frecAcum1.append(lista_puntaje[i]+frecAcum1[-1])
        frecAcum2.append(lista_puntaje2[i]+frecAcum2[-1])
        frecAcum3.append(lista_puntaje3[i]+frecAcum3[-1])
        i+=1

    DepotsParaRuteos=[depots_abiertos, depots_abiertos2, depots_abiertos3]

    L3=(len(hc)**2)/3
    i=0
    while i<L3:
        DepotsParaRuteos.append(AbrirDepotsSemilla(DepotsPorAbrir1,frecAcum1,depotsSemilla1))
        DepotsParaRuteos.append(AbrirDepotsSemilla(DepotsPorAbrir2,frecAcum2,depotsSemilla2))
        DepotsParaRuteos.append(AbrirDepotsSemilla(DepotsPorAbrir3,frecAcum3,depotsSemilla3))
        i+=1

    DepotsParaRuteos=EliminarRepetidos(DepotsParaRuteos)
    #print('\n Seeds for routing:')
    return DepotsParaRuteos

#Assigment first client...
def AsignacionPrimerCliente(O, DepotsA, S, D, K):
    i=0
    Ocopy=copy.deepcopy(O)
    dicc={}
    DepotsAn=copy.deepcopy(DepotsA)
    OaConsiderar=[]
    DepotsaConsiderar=[]
    llav=[]
    cambio=0
    while DepotsAn.count(1)>0:
        DepotsAIndex=DepotsAn.index(1)
        DepotsaConsiderar.append(DepotsAIndex+1)
        OaConsiderar.append(Ocopy[DepotsAIndex])
        dicc['Depot ' + str(DepotsAIndex+1)]=[]
        if 'Depot ' + str(DepotsAIndex+1) not in llav:
            llav.append('Depot ' + str(DepotsAIndex+1))
        DepotsAn[DepotsAIndex]=0

    while [] in dicc.values() and i<K:
        vector=[99999999999999999999999999999999999999999999999999999999999,'', '']
        c=0
        #print('\nOaconsiderar:',dicc.values())
        for o in OaConsiderar:
            if min(o)<vector[0]:
                vector=[min(o), c, o.index(min(o))]
            c+=1
        for depo in OaConsiderar:
            if vector[2]=='':
                for llaves in dicc:
                    if dicc[llaves]==[]:
                        DepotsA[int(llaves[-1])-1]=0
                        cambio=1
                        llav.remove(llaves)
                        dicc.pop(llaves)
                        break
                break
            depo[vector[2]]=99999999999999999999999999999999999999999999999999999999999
        if cambio==0:
            dicc['Depot '+str(DepotsaConsiderar[vector[1]])].append(vector[2])
            if 'Depot '+str(DepotsaConsiderar[vector[1]]) not in llav:
                llav.append('Depot '+str(DepotsaConsiderar[vector[1]]))
        else:
            break
        i+=1
    if cambio==1:
        dicc, llav, DepotsA=AsignacionPrimerCliente(O, DepotsA, S, D, K)
    return dicc, llav, DepotsA

#Penalty for assigned clients
def NulizarClientesYaAsignados(Clientes,Cdd):
    for i in Clientes:
        for d in Cdd:
            for c in d:
                c[i]=99999999999999999999999999999999999999999999999999999999999

#Assign another clients...
def AsignacionOtrosClientes(Cf, nf, Kf, PCf, Qf, df, llav,cas,NumeroDeRutasPorDeposito):
    HaySplitEntreDepots=0
    ListaDemandasAsignadasOtrosDepots=[]
    PromedioClientesVisitadosPorRuta=nf//Kf-1
    if PromedioClientesVisitadosPorRuta>1:
        PromedioClientesVisitadosPorRuta-=1
    Ccopy=copy.deepcopy(Cf)
    CaConsiderarPorDepot=[]
    ClientesYaAsignados=[]
    DemandaDelDeposito=[]
    con=0
    for i in PCf:
        for c in PCf[i]:
            ClientesYaAsignados.append(c)
    if NumeroDeRutasPorDeposito==[]:
        while len(NumeroDeRutasPorDeposito)!=len(llav):
            NumeroDeRutasPorDeposito=[]
            m=0.000001
            for i in PCf:
                mc=0
                if len(PCf[i])/len(ClientesYaAsignados)*Kf-len(PCf[i])/len(ClientesYaAsignados)*Kf//1==0.5:
                    mc=1
                    m*=-1
                NumeroDeRutasPorDeposito.append(round(len(PCf[i])/len(ClientesYaAsignados)*Kf+mc*m))
                if NumeroDeRutasPorDeposito[-1]==0:
                    ClientesSinDeposito=PCf[llav[len(NumeroDeRutasPorDeposito)-1]]
                    for cliente in ClientesSinDeposito:
                        ClientesYaAsignados.remove(cliente)
                    cas[int(llav[len(NumeroDeRutasPorDeposito)-1][-1])-1]=0
                    del PCf[llav[len(NumeroDeRutasPorDeposito)-1]]
                    llav.pop(len(NumeroDeRutasPorDeposito)-1)
                    NumeroDeRutasPorDeposito.pop(-1)
                    break

    while sum(NumeroDeRutasPorDeposito) < Kf:
        NumeroDeRutasPorDeposito[-1]+=1
    for i in PCf:
        CaConsiderarPorDepot.append([])
        DemandaDelDeposito.append(0)
        for c in PCf[i]:
            DemandaDelDeposito[con]+=df[c]
            CaConsiderarPorDepot[con].append(Ccopy[c])
        con+=1
    NulizarClientesYaAsignados(ClientesYaAsignados, CaConsiderarPorDepot)
    con=0
    for i in PCf:
        while len(PCf[i])<PromedioClientesVisitadosPorRuta*NumeroDeRutasPorDeposito[con]:
            vector=[99999999999999999999999999999999999999999999999999999999999, '']
            c=0
            for o in CaConsiderarPorDepot[con]:

                if min(o)<vector[0] and DemandaDelDeposito[con]+df[o.index(min(o))]<=Qf*NumeroDeRutasPorDeposito[con]:
                    vector=[min(o), o.index(min(o))]
                c+=1
            if vector==[99999999999999999999999999999999999999999999999999999999999, '']:
                break
            PCf[i].append(vector[1])
            CaConsiderarPorDepot[con].append(Ccopy[vector[1]])
            ClientesYaAsignados.append(vector[1])
            DemandaDelDeposito[con]+=df[vector[1]]
            NulizarClientesYaAsignados(ClientesYaAsignados, CaConsiderarPorDepot)
        con+=1
        
    while len(S)>len(ClientesYaAsignados):
        vector2=[99999999999999999999999999999999999999999999999999999999999, '', '', '']
        c=0
        for i in PCf:
            for o in CaConsiderarPorDepot[c]:
                if min(o)<vector2[0] and DemandaDelDeposito[c]+df[o.index(min(o))]<=Qf*NumeroDeRutasPorDeposito[c]:
                    vector2=[min(o), i, o.index(min(o)), c]
            c+=1
        if vector2[0]!=99999999999999999999999999999999999999999999999999999999999:
            PCf[vector2[1]].append(vector2[2])
            CaConsiderarPorDepot[vector2[3]].append(Ccopy[vector2[2]])
            ClientesYaAsignados.append(vector2[2])
            DemandaDelDeposito[vector2[3]]+=df[vector2[2]]
            NulizarClientesYaAsignados(ClientesYaAsignados, CaConsiderarPorDepot)
        else:
            for clien in range (0,len(Cf)):
                if clien not in ClientesYaAsignados:
                    c=0
                    for i in PCf:
                        for o in CaConsiderarPorDepot[c]:
                            if o[clien]<vector2[0] and DemandaDelDeposito[c]+df[clien]<=Qf*NumeroDeRutasPorDeposito[c]:
                                vector2=[o[clien], i, clien, c]
                        c+=1
                    if vector2[0]!=99999999999999999999999999999999999999999999999999999999999:
                        PCf[vector2[1]].append(vector2[2])
                        CaConsiderarPorDepot[vector2[3]].append(Ccopy[vector2[2]])
                        ClientesYaAsignados.append(vector2[2])
                        DemandaDelDeposito[vector2[3]]+=df[vector2[2]]
                        ClientesYaAsignados.sort()
                        NulizarClientesYaAsignados(ClientesYaAsignados, CaConsiderarPorDepot)
                    else:
                        HaySplitEntreDepots=1
                        cont=0
                        CapacidadRestante=[]
                        for DemDep in DemandaDelDeposito:
                            CapacidadRestante.append(Qf*NumeroDeRutasPorDeposito[cont]-DemDep)
                            cont+=1
                        cont=0
                        for i in PCf:
                            if cont==CapacidadRestante.index(max(CapacidadRestante)):
                                vector2[1]=i
                                vector2[3]=cont
                            cont+=1
                        vector2[2]=clien
                        PCf[vector2[1]].append(vector2[2])
                        DemandaDelDeposito[vector2[3]]+=CapacidadRestante[vector2[3]]
                        df[vector2[2]]-=CapacidadRestante[vector2[3]]
                        ListaDemandasAsignadasOtrosDepots.append([vector2[2],vector2[3],CapacidadRestante[vector2[3]]])

    while True:
        entro=0
        depos=0
        for depo in PCf:
            if DemandaDelDeposito[depos]>Qf*NumeroDeRutasPorDeposito[depos]:
                trasladado=PCf[depo][-1]
                del PCf[depo][-1]
                DemandaDelDeposito[depos]-=df[trasladado]
                depos2=0
                for dep in PCf:
                    if DemandaDelDeposito[depos2]+df[trasladado]<=Qf*NumeroDeRutasPorDeposito[depos2]:
                        PCf[dep].append(trasladado)
                        DemandaDelDeposito[depos2]+=df[trasladado]
                        entro=1
                        break
                    depos2+=1
                if entro==1:
                    break
            depos+=1
        depos=0
        faltaArreglar=0
        if entro==1:
            for depo in PCf:
                if DemandaDelDeposito[depos]>Qf*NumeroDeRutasPorDeposito[depos]:
                    faltaArreglar=1
                depos+=1
        if faltaArreglar==0:
            break
    return PCf, NumeroDeRutasPorDeposito, llav, cas, HaySplitEntreDepots, ListaDemandasAsignadasOtrosDepots

#Actual customers demand
def DemandaActual(lisClien,dem):
    sumando=0
    for clien in lisClien:
        sumando+=dem[clien]
    return sumando

#Routing function
def Ruteo(n,k,Sa,Ooj,Coj,demanda,Depot,Qi,Costo,SplitEntreDepots,infor,indice):
    i=0
    Sp=[]
    RQi=[]
    guardar=[]
    demanda2=copy.deepcopy(demanda)
    HayUnSplit=0
    while i<k:
        RQi.append(Qi)
        Sp.append([])
        guardar.append([])
        i+=1
    RQi_inicial = copy.deepcopy(RQi)
    if SplitEntreDepots==1:
        if infor!=[]:
            for inf in infor:
                if inf[1]==indice:
                    demanda2[inf[0]]=inf[2]

    while Sa != []:
        guardar2=[]
        if [] in Sp:
            rute=0
            for ruta in Sp:
                if ruta == []:
                    vector=[]
                    for cliente in Sa:
                        if vector == []:
                            vector=[cliente, buscar_info(Ooj,Depot,cliente)]
                        elif buscar_info(Ooj,Depot,cliente) > vector[1]:
                            vector=[cliente,buscar_info(Ooj,Depot,cliente)]
                        else:
                            continue
                    RQi[rute]-=demanda2[vector[0]]
                    Sa.remove(vector[0])
                    ruta.append(vector[0])
                    Costo+=vector[1]
                rute+=1
        if Sa==[]:
            break
        for cliente in Sa:
            for rprima in guardar:
                rprima.clear()
            #print(cliente)
            guardar2.append([])
            r=0
            for ruta in Sp:
                i=-1
                if RQi[r]<=0:
                    guardar[r]=[9999999999999999999999999999999999999999999999999999999999999,0,cliente,r,0]
                    r+=1
                    continue

                while i<len(ruta):
                    i+=1
                    incremento=CalcularCostoIncremento(len(ruta),i,Ooj,Coj,ruta,Depot,cliente)
                    if RQi[r]-demanda2[cliente]<0:
                        incremento*=2
                        HaySplit=1
                    else:
                        HaySplit=0
                    if guardar[r]==[]:
                        guardar[r]=[incremento,i,cliente,r,HaySplit]
                    elif guardar[r][0]>incremento:
                        guardar[r]=[incremento,i,cliente,r,HaySplit]

                r+=1
            diff=0
            menor=9999999999999999999999999999999999999999999999999999999999999
            for ruta in guardar:
                if menor>ruta[0]:
                    menor=ruta[0]
                    rut=ruta[3]
            for ruta in guardar:
                diff+=ruta[0]-menor
            guardar2[-1]=[guardar[rut][0],guardar[rut][1],cliente,diff,rut,guardar[rut][4]]
        clienteMaximo=''
        for cliente in guardar2:
            if clienteMaximo=='':
                clienteMaximo=cliente
            elif clienteMaximo[3]<cliente[3]:
                clienteMaximo=cliente
        if clienteMaximo[5]==0:
            Sp[clienteMaximo[4]].insert(clienteMaximo[1],clienteMaximo[2])
            Costo+=clienteMaximo[0]
            Sa.remove(clienteMaximo[2])
            RQi[clienteMaximo[4]]-=demanda2[clienteMaximo[2]]
        elif clienteMaximo[5]==1:
            Sp[clienteMaximo[4]].insert(clienteMaximo[1],clienteMaximo[2])
            Costo+=clienteMaximo[0]/2
            demanda2[clienteMaximo[2]]-=RQi[clienteMaximo[4]]
            RQi[clienteMaximo[4]]=0
            HayUnSplit=1
        RQi_final = copy.deepcopy(RQi)


    porcentaje_utilizacion = []
    for inicial, final in zip(RQi_inicial, RQi_final):
        porcentaje_utilizacion.append(round(((1 - final / inicial) * 100),2))

    return Sp,Costo,HayUnSplit, porcentaje_utilizacion


#Estimate incremental costs
def CalcularCostoIncremento(m,q,C0,Cn0,ruta,dep,clien):
    if q==0:
        increment=(m+1)*buscar_info(C0,dep,clien)+m*(buscar_info(Cn0,clien,ruta[0])-buscar_info(C0,dep,ruta[0]))
    elif q>0 and q<m:
        l=1
        suma=buscar_info(C0,dep,ruta[0])
        while l<q:
            suma+=buscar_info(Cn0,ruta[l-1],ruta[l])
            l+=1
        increment=suma+(m-q+1)*buscar_info(Cn0,ruta[q-1],clien)+(m-q)*(buscar_info(Cn0,clien,ruta[q])-buscar_info(Cn0,ruta[q-1],ruta[q]))
    else:
        l=1
        suma=buscar_info(C0,dep,ruta[0])
        while l<q:
            suma+=buscar_info(Cn0,ruta[l-1],ruta[l])
            l+=1
        increment=suma+buscar_info(Cn0,ruta[m-1],clien)
    return increment

#Swap function
def Swap(rutta,Costo,MCost,Oa,dep):
    n=len(rutta)
    cambio=1
    while cambio==1:
        cambio=0
        for pos in range(0,n):
            for pos2 in range (0,n):
                delta=0
                cliente1=rutta[pos]
                if pos-pos2>1 or pos-pos2<-1 :
                    if pos==0:
                        clienteme1=dep
                        clientema1=rutta[pos+1]
                        if pos2!=n-1 and pos2!=0:
                            clienteDesplazado=rutta[pos2]
                            clienteDesplazadome1=rutta[pos2-1]
                            clienteDesplazadoma1=rutta[pos2+1]

                            delta-=((n-pos)*buscar_info(Oa,clienteme1,cliente1)+(n-pos-1)*buscar_info(MCost,cliente1,clientema1)+(n-pos2)*buscar_info(MCost,clienteDesplazado,clienteDesplazadome1)+(n-pos2-1)*buscar_info(MCost,clienteDesplazado,clienteDesplazadoma1))
                            delta+=((n-pos)*buscar_info(Oa,clienteme1,clienteDesplazado)+(n-pos-1)*buscar_info(MCost,clienteDesplazado,clientema1)+(n-pos2)*buscar_info(MCost,clienteDesplazadome1,cliente1)+(n-pos2-1)*buscar_info(MCost,cliente1,clienteDesplazadoma1))
                            if delta<0:
                                rutta.pop(pos2)
                                rutta.insert(pos2,cliente1)
                                rutta.pop(pos)
                                rutta.insert(pos,clienteDesplazado)
                                Costo+=delta
                                cambio=1
                        elif pos2==0:
                            clienteDesplazado=rutta[pos2]
                            clienteDesplazadome1=dep
                            clienteDesplazadoma1=rutta[pos2+1]

                            delta-=((n-pos)*buscar_info(Oa,clienteme1,cliente1)+(n-pos-1)*buscar_info(MCost,cliente1,clientema1)+(n-pos2)*buscar_info(Oa,clienteDesplazadome1,clienteDesplazado)+(n-pos2-1)*buscar_info(MCost,clienteDesplazado,clienteDesplazadoma1))
                            delta+=((n-pos)*buscar_info(Oa,clienteme1,clienteDesplazado)+(n-pos-1)*buscar_info(MCost,clienteDesplazado,clientema1)+(n-pos2)*buscar_info(Oa,clienteDesplazadome1,cliente1)+(n-pos2-1)*buscar_info(MCost,cliente1,clienteDesplazadoma1))
                            if delta<0:
                                rutta.pop(pos2)
                                rutta.insert(pos2,cliente1)
                                rutta.pop(pos)
                                rutta.insert(pos,clienteDesplazado)
                                Costo+=delta
                                cambio=1
                        else:
                            clienteDesplazado=rutta[pos2]
                            clienteDesplazadome1=rutta[pos2-1]

                            delta-=((n-pos)*buscar_info(Oa,clienteme1,cliente1)+(n-pos-1)*buscar_info(MCost,cliente1,clientema1)+(n-pos2)*buscar_info(MCost,clienteDesplazado,clienteDesplazadome1)+(n-pos2-1)*0)
                            delta+=((n-pos)*buscar_info(Oa,clienteme1,clienteDesplazado)+(n-pos-1)*buscar_info(MCost,clienteDesplazado,clientema1)+(n-pos2)*buscar_info(MCost,clienteDesplazadome1,cliente1)+(n-pos2-1)*0)
                            if delta<0:
                                rutta.pop(pos2)
                                rutta.insert(pos2,cliente1)
                                rutta.pop(pos)
                                rutta.insert(pos,clienteDesplazado)
                                Costo+=delta
                                cambio=1
                    elif pos!=n-1:
                        clienteme1=rutta[pos-1]
                        clientema1=rutta[pos+1]
                        if pos2!=n-1 and pos2!=0:
                            clienteDesplazado=rutta[pos2]
                            clienteDesplazadome1=rutta[pos2-1]
                            clienteDesplazadoma1=rutta[pos2+1]

                            delta-=((n-pos)*buscar_info(MCost,clienteme1,cliente1)+(n-pos-1)*buscar_info(MCost,cliente1,clientema1)+(n-pos2)*buscar_info(MCost,clienteDesplazado,clienteDesplazadome1)+(n-pos2-1)*buscar_info(MCost,clienteDesplazado,clienteDesplazadoma1))
                            delta+=((n-pos)*buscar_info(MCost,clienteme1,clienteDesplazado)+(n-pos-1)*buscar_info(MCost,clienteDesplazado,clientema1)+(n-pos2)*buscar_info(MCost,clienteDesplazadome1,cliente1)+(n-pos2-1)*buscar_info(MCost,cliente1,clienteDesplazadoma1))
                            if delta<0:
                                rutta.pop(pos2)
                                rutta.insert(pos2,cliente1)
                                rutta.pop(pos)
                                rutta.insert(pos,clienteDesplazado)
                                Costo+=delta
                                cambio=1
                        elif pos2==0:
                            clienteDesplazado=rutta[pos2]
                            clienteDesplazadome1=dep
                            clienteDesplazadoma1=rutta[pos2+1]

                            delta-=((n-pos)*buscar_info(MCost,clienteme1,cliente1)+(n-pos-1)*buscar_info(MCost,cliente1,clientema1)+(n-pos2)*buscar_info(Oa,clienteDesplazadome1,clienteDesplazado)+(n-pos2-1)*buscar_info(MCost,clienteDesplazado,clienteDesplazadoma1))
                            delta+=((n-pos)*buscar_info(MCost,clienteme1,clienteDesplazado)+(n-pos-1)*buscar_info(MCost,clienteDesplazado,clientema1)+(n-pos2)*buscar_info(Oa,clienteDesplazadome1,cliente1)+(n-pos2-1)*buscar_info(MCost,cliente1,clienteDesplazadoma1))
                            if delta<0:
                                rutta.pop(pos2)
                                rutta.insert(pos2,cliente1)
                                rutta.pop(pos)
                                rutta.insert(pos,clienteDesplazado)
                                Costo+=delta
                                cambio=1
                        else:
                            clienteDesplazado=rutta[pos2]
                            clienteDesplazadome1=rutta[pos2-1]

                            delta-=((n-pos)*buscar_info(MCost,clienteme1,cliente1)+(n-pos-1)*buscar_info(MCost,cliente1,clientema1)+(n-pos2)*buscar_info(MCost,clienteDesplazado,clienteDesplazadome1)+(n-pos2-1)*0)
                            delta+=((n-pos)*buscar_info(MCost,clienteme1,clienteDesplazado)+(n-pos-1)*buscar_info(MCost,clienteDesplazado,clientema1)+(n-pos2)*buscar_info(MCost,clienteDesplazadome1,cliente1)+(n-pos2-1)*0)
                            if delta<0:
                                rutta.pop(pos2)
                                rutta.insert(pos2,cliente1)
                                rutta.pop(pos)
                                rutta.insert(pos,clienteDesplazado)
                                Costo+=delta
                                cambio=1
                elif pos<pos2 and pos2-pos==1:
                    if pos==0:
                        clienteme1=dep
                        clientema1=rutta[pos+1]
                        if pos2!=n-1 and pos2!=0:
                            clienteDesplazado=rutta[pos2]
                            clienteDesplazadome1=rutta[pos2-1]
                            clienteDesplazadoma1=rutta[pos2+1]

                            delta-=((n-pos)*buscar_info(Oa,clienteme1,cliente1)+(n-pos2-1)*buscar_info(MCost,clienteDesplazado,clienteDesplazadoma1))
                            delta+=((n-pos)*buscar_info(Oa,clienteme1,clienteDesplazado)+(n-pos2-1)*buscar_info(MCost,cliente1,clienteDesplazadoma1))
                            if delta<0:
                                rutta.pop(pos2)
                                rutta.insert(pos2,cliente1)
                                rutta.pop(pos)
                                rutta.insert(pos,clienteDesplazado)
                                Costo+=delta
                                cambio=1
                        else:
                            clienteDesplazado=rutta[pos2]
                            clienteDesplazadome1=rutta[pos2-1]

                            delta-=((n-pos)*buscar_info(Oa,clienteme1,cliente1)+(n-pos2-1)*0)
                            delta+=((n-pos)*buscar_info(Oa,clienteme1,clienteDesplazado)+(n-pos2-1)*0)
                            if delta<0:
                                rutta.pop(pos2)
                                rutta.insert(pos2,cliente1)
                                rutta.pop(pos)
                                rutta.insert(pos,clienteDesplazado)
                                Costo+=delta
                                cambio=1
                    elif pos!=n-1:
                        clienteme1=rutta[pos-1]
                        clientema1=rutta[pos+1]
                        if pos2!=n-1 and pos2!=0:
                            clienteDesplazado=rutta[pos2]
                            clienteDesplazadome1=rutta[pos2-1]
                            clienteDesplazadoma1=rutta[pos2+1]

                            delta-=((n-pos)*buscar_info(MCost,clienteme1,cliente1)+(n-pos2-1)*buscar_info(MCost,clienteDesplazado,clienteDesplazadoma1))
                            delta+=((n-pos)*buscar_info(MCost,clienteme1,clienteDesplazado)+(n-pos2-1)*buscar_info(MCost,cliente1,clienteDesplazadoma1))
                            if delta<0:
                                rutta.pop(pos2)
                                rutta.insert(pos2,cliente1)
                                rutta.pop(pos)
                                rutta.insert(pos,clienteDesplazado)
                                Costo+=delta
                                cambio=1
                        else:
                            clienteDesplazado=rutta[pos2]
                            clienteDesplazadome1=rutta[pos2-1]

                            delta-=((n-pos)*buscar_info(MCost,clienteme1,cliente1)+(n-pos2-1)*0)
                            delta+=((n-pos)*buscar_info(MCost,clienteme1,clienteDesplazado)+(n-pos2-1)*0)
                            if delta<0:
                                rutta.pop(pos2)
                                rutta.insert(pos2,cliente1)
                                rutta.pop(pos)
                                rutta.insert(pos,clienteDesplazado)
                                Costo+=delta
                                cambio=1
    return rutta, Costo

#Routing cost
def CalcularCostoRuta(rutta,Oa,Ca,dep):
    l=len(rutta)
    i=0
    Cost=0
    while l>0:
        if i==0:
            Cost+=l*buscar_info(Oa,dep,rutta[i])
        else:
            Cost+=l*buscar_info(Ca,rutta[i-1],rutta[i])
        l-=1
        i+=1
    return Cost

#Reinsertion
def reinsercion(ruttta,TotalCost,MCost,Oa,dep):
    l=len(ruttta)
    i=0
    J=range(0,l)
    rutta2=copy.deepcopy(ruttta)
    Costo=CalcularCostoRuta(ruttta,Oa,MCost,dep)
    while i<l:
        for j in J:
            movido=rutta2.pop(i)
            rutta2.insert(j,movido)
            Costo2=CalcularCostoRuta(rutta2,Oa,MCost,dep)
            if Costo2<Costo:
                ruttta=copy.deepcopy(rutta2)
                TotalCost+=Costo2-Costo
                Costo=Costo2
                i=-1
                break
        rutta2=copy.deepcopy(ruttta)
        i+=1
    return ruttta, TotalCost

#Assigment 2...
def Asignacion2 (Oc, DepotsA2, Sc, dc, Kc, Qc):
    Ocopy=copy.deepcopy(Oc)
    dicc2={}
    DepotsAn2=copy.deepcopy(DepotsA2)
    OaConsiderar2=[]
    DepotsaConsiderar2=[]
    llav2=[]
    NumeroDeRutasPorDeposito2=[]
    while DepotsAn2.count(1)>0:
        DepotsAIndex2=DepotsAn2.index(1)
        DepotsaConsiderar2.append(DepotsAIndex2+1)
        OaConsiderar2.append(Ocopy[DepotsAIndex2])
        NumeroDeRutasPorDeposito2.append(1)
        dicc2['Depot ' + str(DepotsAIndex2+1)]=[]
        if 'Depot ' + str(DepotsAIndex2+1) not in llav2:
            llav2.append('Depot ' + str(DepotsAIndex2+1))
        DepotsAn2[DepotsAIndex2]=0
    dcopy=copy.deepcopy(dc)
    while sum(NumeroDeRutasPorDeposito2)<Kc:
        i=0
        for numeroVehiculos in NumeroDeRutasPorDeposito2:
            numeroVehiculos+=round(random.random())
            NumeroDeRutasPorDeposito2[i]=numeroVehiculos
            i+=1
            if sum(NumeroDeRutasPorDeposito2)>=Kc:
                break
    CapacidadUsada=copy.deepcopy(NumeroDeRutasPorDeposito2)
    CapacidadDepot=copy.deepcopy(NumeroDeRutasPorDeposito2)
    i=0
    while i < len(NumeroDeRutasPorDeposito2):
        CapacidadUsada[i]*=0
        CapacidadDepot[i]*=Qc
        i+=1
    ordena=np.argsort(dcopy)
    orden=[]
    #print(ordena)
    for i in ordena:
        orden.append(i)
    i=len(Sc)-1
    while 0<=i:
        cliente=orden[i]
        masCorto=[]
        j=0
        for depot in llav2:
            Dist=buscar_info(Oc,int(depot[-1])-1,cliente)
            if masCorto==[] and dcopy[cliente]+CapacidadUsada[j]<=CapacidadDepot[j]:
                masCorto=[Dist,depot,j]
            elif masCorto==[]:
                masCorto=masCorto
            elif masCorto[0]>Dist and dcopy[cliente]+CapacidadUsada[j]<=CapacidadDepot[j]:
                masCorto=[Dist,depot,j]
            else:
                if masCorto==[]:
                    masCorto.append('ayuda')
            j+=1
        if masCorto==[]:
            break
        CapacidadUsada[masCorto[2]]+=dcopy[cliente]
        dicc2[masCorto[1]].append(cliente)
        i-=1
    return dicc2, llav2, DepotsA2, NumeroDeRutasPorDeposito2

#Perturbation...
def Perturbation(ruta1, ruta2, Costo, MCost, dep, dem, cap):
    n1=len(ruta1)
    n2=len(ruta2)
    #Iterations...
    #nir_ = math.floor(K)
    #nir_ = math.floor(len(C)/3)
    nir_ = math.floor(len(C)/2)
    intentos = 0

    while intentos < nir_:
        
        rem_r1=cap
        rem_r2=cap
        for cliente in ruta1:
            rem_r1-=dem[cliente-1]
        for cliente in ruta2:
            rem_r2-=dem[cliente-1]
        if rem_r1<0 or rem_r2<0:
            return ruta1, ruta2, Costo
        
        temp1 = rem_r1
        temp2 = rem_r2

        clientes_ruta1 = set(ruta1)
        clientes_ruta2 = set(ruta2)

        # Random customer selection on route 1
        if n1 <= 2:
            return ruta1, ruta2, Costo
        else:
            pos1 = random.randrange(1, n1 - 1)  # Avoid start and end of the route
            if ruta1[pos1] in clientes_ruta2:
                intentos += 1
                continue

        if n2 <= 2:
            return ruta1, ruta2, Costo
        else:
            # Random customer selection on route 2
            pos2 = random.randrange(1, n2 - 1)
            if ruta2[pos2] in clientes_ruta1:
                intentos += 1
                continue

        # Check capacity constraints
        if (dem[ruta2[pos2] - 1] <= temp1 and dem[ruta1[pos1] - 1] <= temp2):
            break

        intentos += 1

    if intentos == nir_:
        return ruta1, ruta2, Costo  # Returns routes and cost unchanged

    # Make the exchange
    cliente1 = ruta1[pos1]
    cliente2 = ruta2[pos2]
    clienteme1 = ruta1[pos1 - 1]
    clientema1 = ruta1[pos1 + 1]
    clienteme2 = ruta2[pos2 - 1]
    clientema2 = ruta2[pos2 + 1]

    delta = 0
    delta -= ((n1 - pos1) * buscar_info(MCost, clienteme1, cliente1) +
              (n1 - pos1 - 1) * buscar_info(MCost, cliente1, clientema1) +
              (n2 - pos2) * buscar_info(MCost, clienteme2, cliente2) +
              (n2 - pos2 - 1) * buscar_info(MCost, cliente2, clientema2))
    delta += ((n1 - pos1) * buscar_info(MCost, clienteme1, cliente2) +
              (n1 - pos1 - 1) * buscar_info(MCost, cliente2, clientema1) +
              (n2 - pos2) * buscar_info(MCost, clienteme2, cliente1) +
              (n2 - pos2 - 1) * buscar_info(MCost, cliente1, clientema2))

    ruta2.pop(pos2)
    ruta2.insert(pos2,cliente1)
    ruta1.pop(pos1)
    ruta1.insert(pos1,cliente2)
    Costo += delta

    return ruta1, ruta2, Costo



In [4]:
#================================Main==========================================
recuentoFinalOrdenadoPorInstancia=[]
read_data = pd.read_excel('data_resultados4.xlsx') #name of the excel file where results will be saved..
data_by_save = { 'Instance ID': [], 'Open_depots': [], 'Splits':[], 'Routes': [], 'Utilization_percentage':[], 'Cost': [], 'Time': []}


for Da in Data:
    print(f"\n====================== Instance {Da} ======================\n")

    for iteracion in range(1, 31):
        print(f"\n====================== Iteration {iteracion} ======================\n")
    
        timeStart = time.time()
        O=Da.O
        C=Da.C
        hc=Da.hc
        d=copy.deepcopy(Da.d)
        d2=copy.deepcopy(d)
        K=Da.K
        Q=Da.Q
        D=list(range(0,len(O)))
        S=list(range(0,len(C)))
        data_by_save['Instance ID'].append(Da)
        PropuestaDepots = (depots(O,C,D,hc,d,K,Q,S))
        mejorPropuesta=[]
        for casoApertura in PropuestaDepots:
            d=copy.deepcopy(Da.d)
            if not all(elemento == 0 for elemento in casoApertura):

                CostoTotal=0
                CostoTotal2=0
                casoApertura2=copy.deepcopy(casoApertura)
                PC, llaves, casoApertura=AsignacionPrimerCliente(O, casoApertura, S, D, K)
                PC2, llaves2, casoApertura2, NoLesees=Asignacion2(O,casoApertura,S,d,K,Q)
                casoApertura2=copy.deepcopy(casoApertura2)

                PC, NRPC, llaves, casoApertura, SplitDepots, info=AsignacionOtrosClientes(C, len(S), K, PC, Q, d, llaves, casoApertura, [])
                PC2, NRPC2, llaves2, casoApertura2, SplitDepots2, info2=AsignacionOtrosClientes(C,len(S),K,PC2,Q,d2,llaves2,casoApertura2, NoLesees)

                CasoApertura=copy.deepcopy(casoApertura)
                CasoApertura2=copy.deepcopy(casoApertura2)
                CA=sum(CasoApertura2)
                ji=0
                split=0
                porcentaje_apertura1= []
                while 1 in CasoApertura:
                    rutt, CostoTotal, Split, porcentaje1 = Ruteo(len(PC[llaves[ji]]),NRPC[ji],PC[llaves[ji]],O,C,d,CasoApertura.index(1),Q,CostoTotal,SplitDepots,info,ji)
                    porcentaje_apertura1.extend(porcentaje1)
                    CasoApertura[CasoApertura.index(1)]=0
                    PC[llaves[ji]]=rutt
                    if Split==1:
                        split=1
                    if SplitDepots==1:
                        split=1
                    ji+=1
                    
                if CA>1:
                    ji2=0
                    split2=0
                    porcentaje_apertura2=[]
                    while 1 in CasoApertura2:
                        rutt2, CostoTotal2, Split2, porcentaje2 = Ruteo(len(PC2[llaves2[ji2]]),NRPC2[ji2],PC2[llaves2[ji2]],O,C,d2,CasoApertura2.index(1),Q,CostoTotal2,SplitDepots2,info2,ji2)
                        porcentaje_apertura2.extend(porcentaje2)
                        CasoApertura2[CasoApertura2.index(1)]=0
                        PC2[llaves2[ji2]]=rutt2
                        if Split2==1:
                            split2=1
                        if SplitDepots2==1:
                            split2=1
                        ji2+=1
                k=1
                while k==1:
                    k=0
                    for deposito in PC:
                        irut=0
                        for ruta in PC[deposito]:
                            #print(ruta)
                            LATAM=CostoTotal
                            #print(ruta,'previo')
                            ruta, CostoTotal = reinsercion(ruta,CostoTotal,C,O,int(deposito[-1])-1)
                            ruta, CostoTotal = Swap(ruta,CostoTotal,C,O,int(deposito[-1])-1)
                            PC[deposito][irut]=ruta
                            irut+=1
                            if LATAM!=CostoTotal:
                                k=1
                if CA>1:
                    k2=1
                    while k2==1:
                        k2=0
                        for deposito in PC2:
                            irut2=0
                            for ruta in PC2[deposito]:
                                LATAM2=CostoTotal2
                                ruta, CostoTotal2 = reinsercion(ruta,CostoTotal2,C,O,int(deposito[-1])-1)
                                ruta, CostoTotal2 = Swap(ruta,CostoTotal2,C,O,int(deposito[-1])-1)
                                PC2[deposito][irut2]=ruta
                                irut2+=1
                                if LATAM2!=CostoTotal2:
                                    k2=1
                CostoTotal+=np.dot(casoApertura,hc)
                CostoTotal2+=np.dot(casoApertura2,hc)
                if CA>1:

                    if CostoTotal<CostoTotal2:
                        if mejorPropuesta==[]:
                            mejorPropuesta=[casoApertura,PC,CostoTotal,split]
                            porcentaje = porcentaje_apertura1

                        elif mejorPropuesta[2]>CostoTotal:
                            mejorPropuesta=[casoApertura,PC,CostoTotal,split]
                            porcentaje = porcentaje_apertura1
                    else:
                        if mejorPropuesta==[]:
                            mejorPropuesta=[casoApertura2,PC2,CostoTotal2,split2]
                            porcentaje = porcentaje_apertura2
                        elif mejorPropuesta[2]>CostoTotal2:
                            mejorPropuesta=[casoApertura2,PC2,CostoTotal2,split2]
                            porcentaje = porcentaje_apertura2
                else:
                    if mejorPropuesta==[]:
                        mejorPropuesta=[casoApertura,PC,CostoTotal,split]
                        porcentaje = porcentaje_apertura1
                    elif mejorPropuesta[2]>CostoTotal:
                        mejorPropuesta=[casoApertura,PC,CostoTotal,split]
                        porcentaje = porcentaje_apertura1

        #Perturbation and imporvement phase
        #nmov = math.floor(len(C)**2/4)
        #nmov = math.floor(len(C)**2/2)
        nmov = math.floor(len(C)**2) #Number of iterations
        PLAT = mejorPropuesta[2] #Save better solution at the end
        for it in range(1, nmov):
          # Check if the deposit exists in mejorPropuesta[1]
          for deposito in mejorPropuesta[1]:
            if deposito in mejorPropuesta[1]:
                # Select two different random routes from mejorPropuesta[1][deposito]
                rutas_de_deposito = mejorPropuesta[1][deposito]
                if len(rutas_de_deposito) > 1:
                    ruta1, ruta2 = random.sample(rutas_de_deposito, 2)  # Choose two different routes
                else:
                    # If there is only one route in the depot
                    ruta1 = rutas_de_deposito[0]
                    
                    # Find another depot with at least one route
                    ruta2 = None
                    for otro_deposito, otras_rutas in mejorPropuesta[1].items():
                        if otro_deposito != deposito and len(otras_rutas) > 0:
                            ruta2 = random.choice(otras_rutas)  # Select a route from the other depot
                            break
                            
                    # If no other depot with routes was found, handle the case
                    if ruta2 is None:
                        continue  # Skip this iteration if no routes are available

                # Applies perturbation, reinsertion and swap operations on both paths
                ruta1, ruta2, CostoTotal = Perturbation(ruta1, ruta2, CostoTotal, C, int(deposito[-1]) - 1,d,Q)
                ruta1, CostoTotal = reinsercion(ruta1, CostoTotal, C, O, int(deposito[-1]) - 1)
                ruta1, CostoTotal = Swap(ruta1, CostoTotal, C, O, int(deposito[-1]) - 1)
                ruta2, CostoTotal = reinsercion(ruta2, CostoTotal, C, O, int(deposito[-1]) - 1)
                ruta2, CostoTotal = Swap(ruta2, CostoTotal, C, O, int(deposito[-1]) - 1)
                
                
            else:
                break # Exits the perturbation loop if not found


        data_by_save['Open_depots'].append(str(mejorPropuesta[0]))
        if mejorPropuesta[3]==1:
            data_by_save['Splits'].append('si')
        else:
            data_by_save['Splits'].append('no')
        string_depots_rutas = ''
        for depot in mejorPropuesta[1]:
            for ruta in mejorPropuesta[1][depot]:
                string_aux = "{}: {} ; ".format(depot, ruta)
                string_depots_rutas += string_aux


        data_by_save['Routes'].append(str(string_depots_rutas))
        recuentoFinalOrdenadoPorInstancia.append(mejorPropuesta)
        data_by_save['Cost'].append(str(mejorPropuesta[2]))
        print('\nEnd of instance',Da,'\n=============================================\n\n\n\n')
        data_by_save['Utilization_percentage'].append(str(porcentaje))
        timeEnd = time.time()
        tiempo_transcurrido = timeEnd - timeStart
        data_by_save['Time'].append(str(tiempo_transcurrido))
        pregunta = 1
        if pregunta=='1':
            continue
        elif pregunta=='0':
            exportador = pd.DataFrame(data_by_save)
            df_new = pd.concat([read_data, exportador], ignore_index=True)
            df_new.to_excel('data_resultados4.xlsx', index=False)
            break
        else:
            'incorrectly entered, will continue'
            continue
    print('There are no more cases or they were omitted, end of the race')
    exportador = pd.DataFrame(data_by_save)
    df_new = pd.concat([read_data, exportador], ignore_index=True)
    #Export data
    df_new.to_excel(r"C:\Users\jegr9\OneDrive\Escritorio\DCI\My_papers\Latency-LRP\Heurística\Paper Location-Routing Latency- JMoraga respaldo\data_resultados4.xlsx", index=False)


====================== Instance <module 'coord20_5_1' from 'C:\\Users\\jegr9\\OneDrive\\Escritorio\\DCI\\My_papers\\Latency-LRP\\Heurística\\Paper Location-Routing Latency- JMoraga respaldo\\coord20_5_1.py'> ======================


====================== Iteration 1 ======================


End of instance <module 'coord20_5_1' from 'C:\\Users\\jegr9\\OneDrive\\Escritorio\\DCI\\My_papers\\Latency-LRP\\Heurística\\Paper Location-Routing Latency- JMoraga respaldo\\coord20_5_1.py'> 





====================== Iteration 2 ======================


End of instance <module 'coord20_5_1' from 'C:\\Users\\jegr9\\OneDrive\\Escritorio\\DCI\\My_papers\\Latency-LRP\\Heurística\\Paper Location-Routing Latency- JMoraga respaldo\\coord20_5_1.py'> 





====================== Iteration 3 ======================


End of instance <module 'coord20_5_1' from 'C:\\Users\\jegr9\\OneDrive\\Escritorio\\DCI\\My_papers\\Latency-LRP\\Heurística\\Paper Location-Routing Latency- JMoraga respaldo\\coord20_5_1.py'